In [1]:
import json
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
dong = ['상무1동', '상무2동', '금호1동', '금호2동', '화정1동', '화정2동', '화정3동', '화정4동', '양동', '양3동', '농성1동', '농성2동', '광천동', '유덕동', '치평동', '서창동', '동천동', '풍암동']

dong1 = ['상무1동', '상무2동', '금호1동', '금호2동'] # 남규형
dong2 = ['화정1동', '화정2동', '화정3동', '화정4동'] # 진우형
dong3 = ['양동', '양3동', '농성1동', '농성2동'] # 승현이
dong4 = ['광천동', '유덕동', '치평동', '서창동', '동천동', '풍암동']

bigcode = ["G2", "I2", "S2", "R1", "P1", "L1", "I1", "M1", "Q1", "N1"]

In [ ]:
columns_ = ['구', '업종이름', '검색업종', '23.12', '24.01', '24.02', '24.03', '24.04', 
           '24.05', '24.06', '24.07', '24.08', '24.09', '24.10', '24.11', '24.12']
delivery_sales_df  = pd.DataFrame(columns=columns_)

In [2]:
s = Service(ChromeDriverManager().install())
driver = wb.Chrome(service=s)
driver.maximize_window()
driver.get('https://bigdata.sbiz.or.kr/gis/?type=detail&rptpType=gisDetail&lat=35.1287735&lng=126.8532396')

In [ ]:
error_cate = []
old_dong = ''

In [ ]:
#################################### 반복할 동 선택
seogu = dong3[:]                   # 만약 두번째 동을 시작해야 할 경우 dong[1:] 와 같이 인덱스 입력
####################################
turn = 0                           # 중간에 에러가 났을 경우
#################################### 프린트 되어있는 진행횟수에 적혀있는 숫자 입력

In [ ]:
lst = [
    "김밥/만두/분식",
    "버거",
    "빵/도넛",
    "치킨",
    "토스트/샌드위치/샐러드",
    "카페",
    "마라탕/훠궈",
    "중국집",
    "국/탕/찌개류",
    "국수/칼국수",
    "백반/한정식",
    "전/부침개",
    "해산물 구이/찜",
    "피자",
    "기타 동남아식 전문",
    "베트남식 전문",
    "기타 일식 음식점",
    "일식 카레/돈가스/덮밥",
    "일식 회/초밥",
    "곱창 전골/구이",
    "닭/오리고기 구이/찜",
    "돼지고기 구이/찜",
    "소고기 구이/찜",
    "경양식",
    "파스타/스테이크",
    "패밀리레스토랑"
]

In [ ]:
try:
    for dong in seogu: #### 구 반복 ####
        flag = True
        
        if old_dong != dong: # 동이 바뀌었을 경우
            turn = 0 # 진행도 초기화

        while flag:
            time.sleep(1)
            search_box = driver.find_element(By.ID, "searchAddress")  # 검색창

            time.sleep(1)
            search_box.send_keys(Keys.CONTROL + "a")  # 동 입력창 초기화
            time.sleep(1)
            search_box.send_keys(Keys.DELETE)
            time.sleep(1)

            search_box.send_keys(f"광주 서구 {dong}")  # 입력창에 원하는 동 입력
            search_box.send_keys(Keys.RETURN)
            time.sleep(1)

            button = driver.find_element(By.CSS_SELECTOR, "ul#admListAddress>li>button") # 검색
            
            if button.text == f"광주 서구 {dong}":
                flag = False                              # while 나가기 -> 동 검색 끝
                button.click()
            else:
                print("동 찾기 에러")
                time.sleep(3)

        for i in range(turn, len(lst)):           #### 세부업종 구간 ####
            time.sleep(1)
            Upjong = driver.find_element(By.ID, "selectedUpjong") # 업종 입력창

            Upjong.send_keys(Keys.CONTROL + "a")  # 업종 입력창 초기화
            Upjong.send_keys(Keys.DELETE)
            time.sleep(1)

            up_name = lst

            Upjong.send_keys("> " + up_name)  # 입력창에 업종 검색
            Upjong.send_keys(Keys.RETURN)
            time.sleep(1)

            search = driver.find_elements(By.CSS_SELECTOR, "ul#tpbizSearchListUl>li>button")# 연관 검색 클릭
            for j in range(len(search)):
                if search[j].text.find(up_name) != -1:
                    button = search[j]
                    break
            button.click()
            time.sleep(1)

            button = driver.find_element(By.ID, "analysisBtn")  # 분석하기 버튼 클릭
            button.click()
            time.sleep(8)

            td_name= driver.find_elements(By.CSS_SELECTOR,'div.boxArea>div.summary>dl') # 상세 페이지에 있는 업종 명 가져오기
            for k in range(len(td_name)):                                               #    (제대로 가져왔는지 확인을 위해)
                if td_name[k].text.find(up_name) != -1:
                    td_name = td_name[k].text

            # 배달 매출 데이터 가져오기
            try :                                                            # 배달 매출 txt 바로 리스트로 바꿔 넣기
                delivery_sales_data = driver.find_element(By.CSS_SELECTOR, "tr#genderAllSum").text.split(" ")[2:]
                delivery_sales_df.loc[i] = [dong, up_name, td_name] + delivery_sales_data # 최종 데이터 DF에 추가
            except:
                pass # 배달 매출 정보가 없을 경우 배달을 하지 않는 업종이므로 pass

            # 상세 분석 나가기
            try:
                close = driver.find_element(By.CSS_SELECTOR, "section#reportDiv>div.reportInner>button.close")
                close.click()
            except:
                time.sleep(3)
                close = driver.find_element(By.CSS_SELECTOR, "section#reportDiv>div.reportInner>button.close")
                close.click()
                
            turn = i # 진행도 저장
            old_dong = dong # 진행하고 있는 동 저장
            print(f"{dong}, 진행 횟수: {turn + 1}, 남은 횟수: {len(27)-(turn+1)}")

        ## 저장 후 DF초기화
        delivery_sales_df.to_csv(f"./data/02_배달/{dong}_배달_매출2.csv", encoding='utf-8-sig', index=False)
        delivery_sales_df = pd.DataFrame(columns=columns_)
        
except Exception as e:
    print("에러 발생:", e)

In [ ]:
# 남성 데이터
# g.highcharts-series.highcharts-series-0.highcharts-column-series.highcharts-tracker>rect
# 반복해서
# aria-label="1. 24년01월, 53. 남성.
# 가져오기


# 여성 데이터
# g.highcharts-series.highcharts-series-1.highcharts-column-series.highcharts-tracker>rect
# 반복해서
# aria-label="1. 24년01월, 100. 여성."
# 가져오기

In [ ]:
test = driver.find_element(By.CSS_SELECTOR, "g.highcharts-series.highcharts-series-0.highcharts-column-series.highcharts-tracker>rect")
test

<selenium.webdriver.remote.webelement.WebElement (session="1b7dd725016727792bb53af90045189b", element="f.8262312B683EB886B2F8076A3A048156.d.B9890C827ABC80B7A485695868B23362.e.204")>